In [1]:
!wget 'https://drive.usercontent.google.com/download?id=0Bz8a_Dbh9QhbZVhsUnRWRDhETzA&export=download&authuser=0'

--2024-11-10 14:53:49--  https://drive.usercontent.google.com/download?id=0Bz8a_Dbh9QhbZVhsUnRWRDhETzA&export=download&authuser=0
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.175.132, 2404:6800:4003:c1c::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.175.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2478 (2.4K) [text/html]
Saving to: ‘download?id=0Bz8a_Dbh9QhbZVhsUnRWRDhETzA&export=download&authuser=0’

download?id=0Bz8a_D 100%[===================>]   2.42K  --.-KB/s    in 0s      

2024-11-10 14:53:50 (32.1 MB/s) - ‘download?id=0Bz8a_Dbh9QhbZVhsUnRWRDhETzA&export=download&authuser=0’ saved [2478/2478]



In [2]:
!pip install gdown

In [3]:
# Download the file using gdown
import gdown

# Your file ID from the original link
file_id = '0Bz8a_Dbh9QhbZVhsUnRWRDhETzA'
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'amazon_review_full_csv.tar.gz', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=0Bz8a_Dbh9QhbZVhsUnRWRDhETzA
From (redirected): https://drive.google.com/uc?id=0Bz8a_Dbh9QhbZVhsUnRWRDhETzA&confirm=t&uuid=3e0275f9-b386-400f-8810-165ceb3f37b0
To: /content/amazon_review_full_csv.tar.gz
100%|██████████| 644M/644M [00:17<00:00, 36.7MB/s]


'amazon_review_full_csv.tar.gz'

In [4]:
!tar -xvzf amazon_review_full_csv.tar.gz

amazon_review_full_csv/
amazon_review_full_csv/train.csv
amazon_review_full_csv/test.csv
amazon_review_full_csv/readme.txt


# Import dependencies

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Load dataset

In [6]:
train_data = pd.read_csv('amazon_review_full_csv/train.csv', header=None)
test_data = pd.read_csv('amazon_review_full_csv/test.csv', header=None)

## Train data info

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       int64 
 1   1       object
 2   2       object
dtypes: int64(1), object(2)
memory usage: 68.7+ MB


In [8]:
train_data.tail()

,0,1,2
2999995,1,Don't do it!!,The high chair looks great when it first comes...
2999996,2,"Looks nice, low functionality",I have used this highchair for 2 kids now and ...
2999997,2,"compact, but hard to clean","We have a small house, and really wanted two o..."
2999998,3,Hard to clean!,I agree with everyone else who says this chair...
2999999,1,what is it saying?,not sure what this book is supposed to be. It ...


### Test data info

In [9]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650000 entries, 0 to 649999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       650000 non-null  int64 
 1   1       649974 non-null  object
 2   2       650000 non-null  object
dtypes: int64(1), object(2)
memory usage: 14.9+ MB


In [10]:
test_data.head()

,0,1,2
0,1,mens ultrasheer,"This model may be ok for sedentary types, but ..."
1,4,Surprisingly delightful,This is a fast read filled with unexpected hum...
2,2,"Works, but not as advertised",I bought one of these chargers..the instructio...
3,2,Oh dear,I was excited to find a book ostensibly about ...
4,2,Incorrect disc!,"I am a big JVC fan, but I do not like this mod..."


In [11]:
train_data['sentiment'] = train_data[0].apply(lambda x: 1 if x in [4, 5] else 0)
test_data['sentiment'] = test_data[0].apply(lambda x: 1 if x in [4, 5] else 0)

In [12]:
train_data['text'] = train_data[1].astype(str) + ' ' + train_data[2].astype(str).fillna('')
test_data['text'] = test_data[1].astype(str) + ' ' + test_data[2].astype(str).fillna('')

In [13]:
train_data.drop(columns=[0,1,2], inplace=True)
test_data.drop(columns=[0,1,2], inplace=True)

In [14]:
train_data.head()

,sentiment,text
0,0,more like funchuck Gave this to my dad for a g...
1,1,Inspiring I hope a lot of people hear this cd....
2,1,The best soundtrack ever to anything. I'm read...
3,1,Chrono Cross OST The music of Yasunori Misuda ...
4,1,Too good to be true Probably the greatest soun...


In [15]:
test_data.head()

,sentiment,text
0,0,mens ultrasheer This model may be ok for seden...
1,1,Surprisingly delightful This is a fast read fi...
2,0,"Works, but not as advertised I bought one of t..."
3,0,Oh dear I was excited to find a book ostensibl...
4,0,"Incorrect disc! I am a big JVC fan, but I do n..."


# Train model

In [23]:
model = Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('classifier', LogisticRegression())
])

In [24]:
model.fit(train_data['text'], train_data['sentiment'])


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer', CountVectorizer(stop_words='english')),
                ('tfidf', TfidfTransformer()),
                ('classifier', LogisticRegression())])

In [25]:
result = model.predict(test_data['text'])

In [26]:
# Evaluation
print(classification_report(test_data['sentiment'], result))

              precision    recall  f1-score   support

           0       0.86      0.87      0.87    390000
           1       0.81      0.79      0.80    260000

    accuracy                           0.84    650000
   macro avg       0.83      0.83      0.83    650000
weighted avg       0.84      0.84      0.84    650000



# Test Model

In [27]:
review = "The model is terrible, kinda disappointed with its simple feature compared to the price tag."

print(model.predict([review]))

[0]
